In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)


In [3]:
COLS_DIAGS = ["Capítulo", "Sección", "Categoría", "Descripción"]

In [4]:
df_procesada = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")
df_procesada["FECHA 1º evaluación"] = pd.to_datetime(
    df_procesada["FECHA 1º evaluación"], format="%Y-%m-%d"
)
df_procesada["F NAC"] = pd.to_datetime(df_procesada["F NAC"], yearfirst=True)

edad_primera_consulta = (
    df_procesada["FECHA 1º evaluación"] - df_procesada["F NAC"]
) / np.timedelta64(1, "Y")

df_procesada["EDAD_PRIMERA_CONSULTA"] = edad_primera_consulta
df_procesada["RANGO_ETARIO_PRIMERA_CONSULTA"] = pd.cut(
    df_procesada["EDAD_PRIMERA_CONSULTA"], [0, 15, 20, 25, 40, np.inf]
)


# Analisis para Congenitas

En este analisis se quieren responder obtener los siguientes insights:

- Frecuencia de diagnosticos por anio
- Frecuencia de edad de primera consulta por anio (grupo etario)
- Frecuencia de procedencia por anio
- Frecuencia de sexo por anio

In [18]:
desglose_sociodemografico = ["ANIO_PRIMERA_EVALUACION", "validacion_Region"]

variables_a_analizar = ["validacion", "RANGO_ETARIO_PRIMERA_CONSULTA", "SEXO"]

In [21]:
dict_resultado = {}
for variable in variables_a_analizar:
    nuevo_desglose = desglose_sociodemografico + [variable]
    if variable == "validacion":
        nuevo_desglose.pop()

    resultado = (
        df_procesada.groupby(nuevo_desglose)["validacion"]
        .value_counts()
        .reset_index(name="conteo")
    )

    dict_resultado[variable] = resultado



In [41]:
display(df_procesada["validacion"].value_counts().sum())
display(dict_resultado["RANGO_ETARIO_PRIMERA_CONSULTA"]["conteo"].sum())
display(dict_resultado["SEXO"]["conteo"].sum())
display(dict_resultado["validacion"]["conteo"].sum())

2861

2785

2787

2793

- De los diccionarios obtenidos se puede ver que existen diferencias en las cantidades totales
de casos. Esto se debe a valores faltantes en cada una de las variables agregadas (ANIO_PRIMERA_EVALUACION,
Region, Rango Etario y Sexo)